## Script for creating priors matrix

This is used for converting the gmt file downloaded from Broad here into priors:
http://software.broadinstitute.org/gsea/msigdb/collections.jsp
Priors are either 1, -1, or, most likely, 0 (it's a sparse network)

To run this , I had to downoad the msigDB full 5.1 gmt file, and replace certain problematic genes with their more common names: ACCN1 with ASIC1, PIDD with PIDD1. 

In [216]:
import csv
import itertools

inferelator_file = "../../Dropbox (Simons Foundation)/Priors_For_Inferelator.csv"
msigdb_file = "/Users/ndeveaux/Downloads/msigdb.v5.1.symbols.gmt"
stop_text = 'The following sets were evaluated, but determined to be lacking:'
HOMfile = '/Users/ndeveaux/Dev/ThirdParty/mu2hu/HOM_MouseHumanSequence.rpt'


In [217]:
all_targets = {}
selected_targets_plus = {}
selected_targets_minus = {}

In [218]:
""" Thanks Emily Miraldi for this code snippet taken from parseMouseGeneHomologs.py

This follows the Jax MGI database instructions:  
http://www.informatics.jax.org/faq/ORTH_dload.shtml
to download a .rpt file that contains mouse - human ortholog mappings

"""

muhuDic = dict() # keys = mouse, item = list of human genes
humuDic = dict() # keys = human, item = list of mouse genes

orthIn = open(HOMfile,'r')
orthNums = list()
muList = list()
huList = list()		
for line in itertools.islice(orthIn,1,None):
	lineInf = list()
	lineInf = line.strip('\n').split('\t')
	orthNum = lineInf[0]
	species = lineInf[1]
	symbol = lineInf[3]
	if orthNum not in orthNums: # first line for this ortholog
		for gene in muList:
			muhuDic[gene] = huList
		for gene in huList:
			humuDic[gene] = muList
		muList = list()
		huList = list()		
		orthNums.append(orthNum)
	if species == 'human':
		huList.append(symbol)
	else:
		muList.append(symbol)

In [219]:
def convert_human_to_mouse(target, organism):
    if target not in muhuDic:
        # print target + ' not found in mouse dictionary'
        if target not in humuDic:
            print target + ' not found in human dictionary either. Aborting'
            # return null if this is a human geneset and an ortholog can't be found
            if organism == 'H':
                print 'unpa' + 'ralleled human gene'
                return
            return target
        else:
            mouse_equivalent = humuDic[target]
            if len(mouse_equivalent) >= 1:
                return mouse_equivalent[0]
            else:
                print 'unusual mouse equivalent, length not 1: ' + str(mouse_equivalent)
                return target
                

In [220]:
def convert_row_to_target(row):
    geneset_id = row[0]
    transcription_factor = row[2].title()
    notes = row[8]
    organism = row[5]
    # A few assumptions need to be met in order to populate the table: not a KnockIn, double TF, or non-Up regulated
    if geneset_id[-3:] == '_UP':
        if 'KnockIn' not in notes:
            if '&' not in transcription_factor:
                # create union of past sets. 
                targets = [convert_human_to_mouse(x, organism) for x in all_targets[geneset_id]]
                print 'UP found: ' + geneset_id
                if targets != None:
                    if transcription_factor in selected_targets_minus:
                        selected_targets_minus[transcription_factor] = targets + selected_targets_minus[transcription_factor]
                    else:
                        selected_targets_minus[transcription_factor] = targets
                down_id = geneset_id.replace('UP', 'DN')
                if down_id in all_targets:
                    print 'DOWN found: ' + down_id
                    targets = [convert_human_to_mouse(x, organism) for x in all_targets[down_id]]
                    if targets != None:
                        if transcription_factor in selected_targets_plus:
                            selected_targets_plus[transcription_factor] = selected_targets_plus[transcription_factor] + targets
                        else:
                            selected_targets_plus[transcription_factor] = targets
                else:
                    print 'missing down-regulated KO geneset for geneset_id: {}'.format(geneset_id)
            else:
                print 'Double KO, & symbol, found in notes for {}'.format(geneset_id)
        else:
            print 'KnockIn found in notes for {}'.format(geneset_id)
    else:
        print 'unexpected format in geneset_id: {} missing UP'.format(geneset_id)

In [221]:
with open(msigdb_file) as f:
    g = f.read() 
    for line in g.split('\n'):
        tab_split = line.split('\t')
        all_targets[tab_split[0]] = tab_split[2:]

In [222]:
with open(inferelator_file, 'rb') as csvfile:
    f = csv.reader(csvfile, delimiter = ',')
    for row in f:
        if row[0] == stop_text:
            break
        convert_row_to_target(row)

unexpected format in geneset_id: ID missing UP
unusual mouse equivalent, length not 1: []
C15orf2 not found in human dictionary either. Aborting
unusual mouse equivalent, length not 1: []
PROL1 not found in human dictionary either. Aborting
PSG5 not found in human dictionary either. Aborting
unusual mouse equivalent, length not 1: []
unusual mouse equivalent, length not 1: []
unusual mouse equivalent, length not 1: []
unusual mouse equivalent, length not 1: []
unusual mouse equivalent, length not 1: []
unusual mouse equivalent, length not 1: []
CST4 not found in human dictionary either. Aborting
unusual mouse equivalent, length not 1: []
UP found: ATM_DN.V1_UP
DOWN found: ATM_DN.V1_DN
unusual mouse equivalent, length not 1: []
LOC51190 not found in human dictionary either. Aborting
unusual mouse equivalent, length not 1: []
C17orf88 not found in human dictionary either. Aborting
unusual mouse equivalent, length not 1: []
POM121L9P not found in human dictionary either. Aborting
unusual 

In [223]:
all_targets = set([item for sublist in selected_targets_minus.values() + selected_targets_plus.values() for item in sublist])

In [224]:
with open('TF_targets_list.csv', 'wb') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',')
    spamwriter.writerow([''] + [key for key in selected_targets_plus])
    for item in all_targets:
        row = [item]
        for key in set(selected_targets_plus.keys() + selected_targets_minus.keys()):
            value = [0, 0]
            if key in selected_targets_plus:
                value[0] = selected_targets_plus[key].count(item)
            if key in selected_targets_minus:
                value[1] = -1 * selected_targets_minus[key].count(item)
            if not all(value):
                value = [0]
            row.append(','.join(map(str, value)))
        spamwriter.writerow(row)
    

In [226]:
with open('TF_targets_sum.csv', 'wb') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',')
    spamwriter.writerow([''] + [key for key in selected_targets_plus])
    for item in all_targets:
        row = [item]
        for key in set(selected_targets_plus.keys() + selected_targets_minus.keys()):
            value = [0, 0]
            if key in selected_targets_plus:
                value[0] = selected_targets_plus[key].count(item)
            if key in selected_targets_minus:
                value[1] = -1 * selected_targets_minus[key].count(item)
            if item == 'Wls' and value[1] == -2:
                import pdb; pdb.set_trace()
            row.append(str(sum(value)))
        spamwriter.writerow(row)
    

> <ipython-input-226-799dda406478>(14)<module>()
-> row.append(str(sum(value)))
(Pdb) l
  9  	                value[0] = selected_targets_plus[key].count(item)
 10  	            if key in selected_targets_minus:
 11  	                value[1] = -1 * selected_targets_minus[key].count(item)
 12  	            if item == 'Wls' and value[1] == -2:
 13  	                import pdb; pdb.set_trace()
 14  ->	            row.append(str(sum(value)))
 15  	        spamwriter.writerow(row)
 16  	    
[EOF]
(Pdb) str(sum(value))
'-1'
(Pdb) item
'Wls'
(Pdb) key
'Pparg'
(Pdb) c
